**Classification and Regression Trees**

* Algorithm that basically makes decisions by creating a tree where each node represents a condition, each edge is a decision on that specific condition, and the leaf is the overall classification/regression output
* Recursive binary splitting is used to find the best splits by measuring the cost of each split
* Two potential cost functions:

$$\sum_{i=1}^{n} (y_i - prediction)^2 $$

or

$$\sum_{i=1}^{k} (p_i * (1 - p_i))$$

where k is the number of labels, and $p_i$ is the proportion of class inputs that are labelled i in the group.


* Tree controls: can set minimum number of inputs to use on each leaf, maximum depth of tree, and pruning (removing branches with low-importance features)


https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052

In [19]:
import numpy as np
import random

training_data = np.array([
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
])

In [3]:
class DecisionNode:
    
    #Initialization
    def __init__(self, data):
        self.data = data
        self.terminal = False
    
    #Printing for debugging
    def dataToString(self):
        print(self.data)
        
    #Split node and create child nodes
    def split(self):
        (group_1, group_2, self.condition, self.feature, gi) = find_best_partition(self.data)
        
        if len(group_1) == 0 or len(group_2) == 0:
            self.terminal = True
            self.prediction = max(set([row[-1] for row in self.data]), key = [row[-1] for row in self.data].count)
        else:
            self.left = DecisionNode(group_1)
            self.right = DecisionNode(group_2)
            self.left.split()
            self.right.split()
    
    def predict(self, test_data):
        if self.terminal == True:
            return self.prediction
        else:     
            if isinstance(test_data[self.feature], str):
                if test_data[self.feature] == self.condition:
                    return self.left.predict(test_data)
                else:
                    return self.right.predict(test_data)
            else:
                if test_data[self.feature] <= self.condition:
                    return self.left.predict(test_data)
                else:
                    return self.right.predict(test_data)

In [4]:
#Partitions data depending on condition
def partition(data, feature, condition):
    group_1 = [];
    group_2 = [];
    
    if isinstance(data[0][feature], str):
        for row in data:
            if row[feature] == condition:
                group_1.append(row)
            else:
                group_2.append(row)
    else:
        for row in data:
            if row[feature] <= condition:
                group_1.append(row)
            else:
                group_2.append(row)
    return group_1, group_2


#calculates Gini score of a group
def gini_score(data):
    classifications = [row[-1] for row in data]
    num = len(classifications)
    if num == 0:
        return 1
    conditions = list(set(classifications))
    sum_ = 0
    for i in conditions:
        proportion = classifications.count(i)/num
        sum_ = sum_ + proportion * (1 - proportion)
    return sum_


#find the best split
def find_best_partition(data):
    best_gini = 999;
    
    for i in range(len(data[0])-1):
        possible_conditions = list(set([row[i] for row in data]))
        for j in possible_conditions:
            (group_1, group_2) = partition(data, i, j)
            #if len(group_1) == 0 or len(group_2) == 0:
            #    continue
            
            gini = gini_score(group_1) + gini_score(group_2)
            if gini < best_gini:
                best_gini = gini
                best_group_1 = group_1
                best_group_2 = group_2
                best_condition = j
                best_feature = i
    return (best_group_1, best_group_2, best_condition, best_feature, best_gini)

def build_tree(data, max_depth, min_size):
    root = DecisionNode(data)
    root.split()
    return root

In [20]:
mushroom_data = np.loadtxt('agaricus-lepiota.data', dtype = 'str', delimiter=',')
mushroom_data = np.delete(mushroom_data, 11, axis = 1)
mushroom_data[:,[0,21]] = mushroom_data[:,[21,0]]

In [53]:
training_set = [];
test_set = [];

for row in mushroom_data:
    if random.random() < 0.001:
        training_set.append(row)
    else:
        test_set.append(row)            

In [54]:
dec_tree = build_tree(training_set, 5, 5)

In [55]:
test_set_no_label = np.delete(test_set, 21, axis = 1)
test_set_labels = np.array(test_set)[:,21]
results = [];

for row in test_set_no_label:
    results.append(dec_tree.predict(row))

results = np.array(results)

count = 0
for i in range(len(results)):
    if results[i] == test_set_labels[i]:
        count += 1

accuracy = count/len(results)
print(accuracy)

0.7111521873074553


['e' 'e' 'e' ... 'e' 'p' 'e']
